In [5]:
using GamsStructure
using JuMP
using Complementarity
using GLPK
using PATHSolver

In [4]:
GU = GamsUniverse()


@GamsSets(GU,"ex01_data",begin
    :i, "Canning plants"
    :j, "Markets"
end)


@GamsParameters(GU,"ex01_data",begin
    :a, (:i,), "Capacity of plant i in cases"
    :b, (:j,), "Demand at market j in cases"
    :d, (:i,:j), "distance in thousands of miles", [1,2] #notice the file d.csv has the wrong columns
end)

@GamsParameters(GU,begin
    :c, (:i,:j), "transport cost in thousands of dollars per case"
end)

@GamsScalars(GU,begin
    :f, 90, "freight in dollars per case per thousand miles"
end)


"""
In this block, we use the code

f = scalar(GU[:f])

as an example of extracting a scalar value. This isn't necessary in 
this code, it works to use GU[:f] in place of f in the following line.

However, if used in a JuMP model, extracting this information is necessary
as JuMP doesn't recognize the datatype and will throw and error.
"""
GU[:c][:i,:j] = scalar(GU[:f])*GU[:d][:i,:j]/1000
GU



Sets

j => Markets
i => Canning plants

Parameters

a => (:i,) => Capacity of plant i in cases
b => (:j,) => Demand at market j in cases
d => (:i, :j) => distance in thousands of miles
c => (:i, :j) => transport cost in thousands of dollars per case

Scalars

f => 90 => freight in dollars per case per thousand miles


In [29]:
function transport_model(GU)
    I = GU[:i]
    J = GU[:j]

    c = GU[:c]
    a = GU[:a]
    b = GU[:b]

    m = MCPModel()

    @variables(m, begin
        w[I] >= 0
        p[J] >= 0
        x[I,J] >= 0
    end)


    @mapping(m,profit[i = I,j=J], w[i]+c[[i],[j]]-p[j])
    @mapping(m,supply[i = I], a[[i]] - sum(x[i,j] for j∈J))
    @mapping(m,demand[j=J], sum(x[i,j] for i∈I) - b[[j]])

    @complementarity(m,profit,x)
    @complementarity(m,supply,w)
    @complementarity(m,demand,p)


    return m

end


transport_model (generic function with 1 method)

In [35]:
function transport_model_mcp(GU)
    I = GU[:i]
    J = GU[:j]

    c = GU[:c]
    a = GU[:a]
    b = GU[:b]

    m = Model(PATHSolver.Optimizer)

    @variables(m, begin
        w[I] >= 0
        p[J] >= 0
        x[I,J] >= 0
    end)

    @constraints(m, begin
        profit[i = I,j=J],  w[i]+c[[i],[j]] - p[j]          ⟂ x[i,j]
        supply[i = I],      a[[i]] - sum(x[i,j] for j∈J)    ⟂ w[i]
        demand[j=J],        sum(x[i,j] for i∈I) - b[[j]]    ⟂ p[j]
    end)

    return m

end


transport_model_mcp (generic function with 1 method)

In [34]:
model = transport_model_mcp(GU)

set_silent(model)

optimize!(model)

value.(model[:x])


2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:seattle, :san_diego]
    Dimension 2, [:new_york, :chicago, :topeka]
And data, a 2×3 Matrix{Float64}:
 0.0  0.0  0.0
 0.0  0.0  0.0

In [32]:
model = transport_model(GU)

solveMCP(model)

Path 5.0.03 (Fri Jun 26 10:05:33 2020)
Written by Todd Munson, Steven Dirkse, Youngdae Kim, and Michael Ferris

Crash Log
major  func  diff  size  residual    step       prox   (label)
    0     0             1.0416e+03             0.0e+00 (demand[new_york])
    1     1     9     3 1.0029e+03  1.0e+00    1.0e+01 (demand[new_york])
    2     2     6     9 9.4289e+02  1.0e+00    9.0e+00 (demand[new_york])
    3     3     0     9 8.8483e+02  1.0e+00    8.1e+00 (demand[new_york])
pn_search terminated: no basis change.

Major Iteration Log
major minor  func  grad  residual    step  type prox    inorm  (label)
    0     0     4     4 8.8483e+02           I 7.3e+00 4.9e+02 (demand[new_york)
    1     1     5     5 7.2119e+02  1.0e+00 SO 2.9e+00 2.6e+02 (profit[seattle,)
    2     2     6     6 3.6245e+02  1.0e+00 SO 1.2e+00 1.9e+02 (profit[san_dieg)
    3     2     7     7 5.9071e+01  1.0e+00 SO 4.7e-01 3.3e+01 (supply[san_dieg)
    4     1     8     8 1.6118e+01  1.0e+00 SO 1.9e-01 7.7e+00 (

:Solved

In [24]:
result_value.(model[:x])

2-dimensional DenseAxisArray{Float64,2,...} with index sets:
    Dimension 1, [:seattle, :san_diego]
    Dimension 2, [:new_york, :chicago, :topeka]
And data, a 2×3 Matrix{Float64}:
  50.0  300.0    0.0
 275.0    0.0  275.0